# Draft

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Settings

In [2]:
# Bitcoin/Thugz League
TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 5/8,          # focus for everyone else
    'assists': 1, 
    'plus_minus': 1/8,     # bad correlation
    'powerplay_points': 1, 
    'shots_on_goal': 1, 
    'hits': 1,
    'blocks': 1
}

GOALIE_CATEGORIES = {
    "wins" : 1,            # good team is good
    "save_percentage": 1/2, 
    "saves": 5/8, 
    "shutouts": 1/2        # can't predict
}

### Players

In [3]:
pic = pd.read_csv("../data/adp-yahoo_com.csv")
cap = pd.read_csv("../data/info-capfriendly_com.csv")
players = pd.merge(pic, cap, how="left", on="name")

players["adp"] = players["pick"] 
players = players[['team', 'age', 'name', 'position', 'adp']]

players.head()

,team,age,name,position,adp
0,Edm,23.0,Connor McDavid,C,1.6
1,Col,25.0,Nathan MacKinnon,C,2.5
2,Edm,25.0,Leon Draisaitl,"C,LW",3.6
3,NYR,29.0,Artemi Panarin,LW,5.2
4,Was,35.0,Alex Ovechkin,LW,5.5


### Multiple Positions

In [4]:
multi = (
    players
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

multi.head()

,name,position
0,Connor McDavid,C
1,Nathan MacKinnon,C
2,Leon Draisaitl,C
3,Leon Draisaitl,LW
4,Artemi Panarin,LW


### Projection Data

In [5]:
proj = pd.read_csv("../data/projections-generated.csv")

### Goalies

In [6]:
goalies = proj[proj["position"] == "G"].copy()

goalies = (
    goalies
    [["name"] + list(GOALIE_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

for key, value in GOALIE_CATEGORIES.items():
    goalies[key] *= value
    
goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= sum(GOALIE_CATEGORIES.values())
goalies["rollup"] *= 100

goalies = goalies.reset_index()
goalies = goalies[["name", "rollup"]]

goalies.head()

,name,rollup
0,Andrei Vasilevskiy,96.058195
1,Connor Hellebuyck,77.919895
2,Tuukka Rask,66.178592
3,Carter Hart,64.573863
4,Robin Lehner,69.378325


### Skaters

In [7]:
skaters = proj[proj["position"] != "G"].copy()
skaters = (
    skaters
    [["name"] + list(SKATER_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)
     
for key, value in SKATER_CATEGORIES.items():
    skaters[key] *= value
    
skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= sum(SKATER_CATEGORIES.values())
skaters["rollup"] *= 100

skaters = skaters.reset_index()
skaters = skaters[["name", "rollup"]]

skaters.head()

,name,rollup
0,Connor McDavid,61.869332
1,Nathan MacKinnon,63.517697
2,Leon Draisaitl,58.949771
3,Artemi Panarin,48.720453
4,Alex Ovechkin,59.346880


### Merge

In [8]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
rollup.head()

,name,position,rollup
7,Andrei Vasilevskiy,G,96.058195
17,Connor Hellebuyck,G,77.919895
45,Jordan Binnington,G,72.210057
38,Robin Lehner,G,69.378325
61,Frederik Andersen,G,69.304398


### VORP

In [9]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in ROSTER.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * TEAMS)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head()

,name,position,rollup,vorp
7,Andrei Vasilevskiy,G,96.058195,33.744156
17,Connor Hellebuyck,G,77.919895,15.605856
45,Jordan Binnington,G,72.210057,9.896018
38,Robin Lehner,G,69.378325,7.064285
61,Frederik Andersen,G,69.304398,6.990359


### Draft

In [16]:
draft = pd.merge(players, vorp, how="left", on="name", suffixes=("_yahoo", ""))
draft = draft.dropna()
draft = draft.groupby("name").head(1).dropna()

draft = draft[[
    'team', 'age', 'name', 
    'position_yahoo', 'position',
    'rollup', 'vorp', 'adp',
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)

draft['pr_vorp'] = draft.groupby("position")['vorp'].rank(ascending=False)
draft['pr_adp'] = draft.groupby("position")['adp'].rank(ascending=True)

draft = draft.sort_values("adp", ascending=True)

In [18]:
draft["pr_arbitrage"] =  draft['pr_adp'] - draft['pr_vorp']
draft['round'] = draft["adp"] // TEAMS + 1

In [19]:
draft.sort_values("pr_arbitrage", ascending=False)

,team,age,name,position_yahoo,position,rollup,vorp,adp,pr_vorp,pr_adp,pr_arbitrage,round
323,Van,34.0,Alexander Edler,D,D,46.0,4.3,173.6,10.0,58.0,48.0,15.0
342,Edm,32.0,Mikko Koskinen,G,G,54.6,-7.8,176.8,14.0,38.0,24.0,15.0
166,Edm,25.0,Darnell Nurse,D,D,44.6,2.9,136.0,11.0,31.0,20.0,12.0
336,LA,36.0,Dustin Brown,RW,RW,36.1,-7.9,175.2,19.0,38.0,19.0,15.0
325,Ari,22.0,Jakob Chychrun,D,D,34.9,-6.8,174.2,40.0,59.0,19.0,15.0
312,VGK,33.0,Alec Martinez,D,D,36.6,-5.1,171.6,37.0,54.0,17.0,15.0
260,Min,33.0,Cam Talbot,G,G,55.5,-6.9,167.2,12.0,29.0,17.0,14.0
324,VGK,27.0,William Karlsson,C,C,34.7,-13.5,173.8,33.0,50.0,17.0,15.0
344,Anh,27.0,Rickard Rakell,"LW,RW",LW,36.6,-9.7,178.8,22.0,37.0,15.0,15.0
164,LA,31.0,Drew Doughty,D,D,43.3,1.6,134.3,14.0,29.0,15.0,12.0


In [20]:
pd.DataFrame(draft.groupby(["round"])['position'].value_counts())

position
round position          
1.0   C                6
      LW               2
      G                1
      RW               1
2.0   C                3
      D                2
      LW               2
      RW               2
      G                1
3.0   LW               3
      C                2
      G                2
      D                1
      RW               1
4.0   C                5
      RW               3
      D                1
      G                1
      LW               1
5.0   D                2
      G                2
      LW               1
6.0   C                4
      D                3
      G                2
      LW               2
      RW               2
7.0   C                2
      G                2
      LW               2
      D                1
      RW               1
8.0   D                4
      RW               3
      C                1
9.0   D                3
      LW               3
      G                2
      RW               1
10.0  D                5
      C                2
      G                2
      RW               1
11.0  D                6
      G                3
      RW               3
      LW               2
      C                1
12.0  C                5
      D                5
      LW               1
13.0  D                4
      C                3
      RW               3
      LW               1
14.0  G               11
      LW              11
      RW              10
      C                5
      D                5
15.0  D               18
      C               13
      G                9
      RW               9
      LW               6
16.0  G                2
      LW               1
      RW               1

### Export

In [21]:
draft.to_csv("../data/draft-thugz_league.csv", index=False)